In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
!which conda

/usr/local/bin/conda


In [ ]:
!conda --version

conda 4.5.4


In [ ]:
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes

In [ ]:
!conda --version

conda 4.10.3


In [ ]:
import sys
sys.path

In [ ]:
!conda env create -f environment.yml

In [ ]:
%%bash 
download.sh

In [ ]:
def obj_cnt(dataloader, args):
    counts = {}
    all_categories = dataloader.dataset.categories
    for i in range(len(all_categories)):
        counts["{0}-{1}".format(i, i)] = 0
        for j in range(i+1, len(all_categories)):
            counts["{0}-{1}".format(i, j)] = 0

    groupings_size = [[] for i in range(len(dataloader.dataset.supercategories_to_names))]
    groupings_dist = [[] for i in range(len(dataloader.dataset.supercategories_to_names))]
    group_mapping = dataloader.dataset.group_mapping
    img_center = np.array([.5, .5])
    instances_size = [[] for i in range(len(all_categories))]

    filepaths = [[] for i in range(len(dataloader.dataset.supercategories_to_names))] # for qualitative examples in analysis step

    with_people_instances = np.zeros(len(all_categories))
    with_people = np.zeros(len(dataloader.dataset.supercategories_to_names))
    not_with_people = np.zeros(len(dataloader.dataset.supercategories_to_names))
    if hasattr(dataloader.dataset, 'people_labels'):
        people = dataloader.dataset.people_labels
    else:
        people = None

    overlap = {}
    overlap_percent = .95

    for i, (data, target) in enumerate(tqdm(dataloader)):
        anns = target[0]
        filepath = target[3]
        categories = list(set([ann['label'] for ann in anns]))
        if people is not None:
            has_people = False
            if len(list(set(people) & set(categories))) > 0:
                has_people = True

        co_added = []
        overlap_added = []
        people_added = []
        sizes_added = []
        for a in range(len(anns)):
            
            bbox = anns[a]['bbox']
            size = (bbox[1]-bbox[0])*(bbox[3]-bbox[2])
            if anns[a]['label'] not in sizes_added:
                instances_size[all_categories.index(anns[a]['label'])].append([size, filepath])
                sizes_added.append(anns[a]['label'])
            elif instances_size[all_categories.index(anns[a]['label'])][-1][0] < size:
                instances_size[all_categories.index(anns[a]['label'])][-1][0] = size
            if group_mapping is not None:
                # size of object and distance of object from image center
                group = group_mapping(anns[a]['label'])
                obj_center = np.array([bbox[0] + (bbox[1]/2.), bbox[2] + (bbox[3]/2.)])
                distance = np.linalg.norm(obj_center - img_center)
                groupings_size[group].append(size)
                groupings_dist[group].append(distance)

                # if there's a person in this image or not
                if people is not None:
                    if group not in people_added:
                        if len(filepaths[group]) < 500:
                            filepaths[group].append(filepath)

                        if has_people:
                            with_people[group] += 1
                        else:
                            not_with_people[group] += 1
                        people_added.append(group)

            # instance and cooccurrence counts
            cat_a = dataloader.dataset.categories.index(anns[a]['label'])
            key = '{0}-{1}'.format(cat_a, cat_a)
            if key not in co_added:
                co_added.append(key)
                counts[key] += 1
                if people is not None and has_people:
                    with_people_instances[cat_a] += 1
            for b in range(a+1, len(anns)):
                cat_b = dataloader.dataset.categories.index(anns[b]['label'])
                if cat_a < cat_b:
                    key = "{0}-{1}".format(cat_a, cat_b)
                else:
                    key = "{0}-{1}".format(cat_b, cat_a)
                if 'bbox' in anns[a].keys() and bb_intersection_over_union(anns[a]['bbox'], anns[b]['bbox']) > overlap_percent and anns[a]['label'] != anns[b]['label']:
                    if key not in overlap_added:
                        overlap_added.append(key)
                        if key in overlap.keys():
                            overlap[key] += 1
                        else:
                            overlap[key] = 1
                if key not in co_added:
                    co_added.append(key)
                    counts[key] += 1

    stats = {}
    stats['counts'] = counts
    stats['overlap'] = overlap
    stats['sizes'] = groupings_size
    stats['distances'] = groupings_dist
    stats['with_people'] = with_people
    stats['not_with_people'] = not_with_people
    stats['with_people_instances'] = with_people_instances
    stats['filepaths'] = filepaths
    stats['instances_size'] = instances_size
    pickle.dump(stats, open("results/{}/obj_cnt.pkl".format(args.folder), "wb"))